# Obtaining a scoped API Token With STAC Privileges

In this tutorial, we will request an access token from the platform API, and then request a scoped API token with STAC privileges from the [create STAC token](https://api.carbonmapper.org/api/v1/docs#/Account/account_api_token_tokens_create_stac) resource.

Resource access is granted based on token scopes. While some resources are entirely public and others private, some others return different results depending on the authorization afforded by the token scope and the group that the account belongs to. The default scopes provided to a newly registered account are _STAC_ and _catalog:read_, and will belong to the _Public_ group. Information about public, catalog, and STAC resources can be found in the [platform API documentation](https://api.carbonmapper.org/).

The scoped API token is only provided to the user once, upon generation. If generating the token for future use, it should be stored securely and should not be committed to code repositories.

Users with Carbon Mapper platform accounts may obtain a scoped token. Users without accounts can [register](https://platform.carbonmapper.org/account/register/) on the Carbon Mapper platform website. 

## Installing Requirements

For tutorial purposes, we will use IPython's [pip](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-pip) magic command to install the [Requests](https://pypi.org/project/requests/) requirement in the current kernel.

In [ ]:
%pip install requests

## Requesting a Scoped Token

In [ ]:
import getpass
import json
from datetime import datetime, timedelta

import requests

base_url = "https://api.carbonmapper.org/api/v1/"

# Request a platform API access token
account_token_path = "token/pair"
account_email = input("Enter the email address associated with your Carbon Mapper account: ")
account_password = getpass.getpass("Enter the password associated with your Carbon Mapper account: ")
payload = {
    "email": account_email,
    "password": account_password,
}

response = requests.post(
    f"{base_url}{account_token_path}",
    json=payload,
)
response.raise_for_status()

access_token = response.json()["access"]

# Request a scoped token with STAC privileges
stac_token_path = "account/tokens/create-stac"
stac_token_name = input("Enter the token name: ")
# The token should expire one week from today
expiration_date = datetime.now() + timedelta(days=7)
payload = {
    "expiration_date": expiration_date.strftime("%Y-%m-%d"),
    "name": stac_token_name,
}

response = requests.post(
    f"{base_url}{stac_token_path}",
    json=payload,
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()

# Assign scoped token to a variable 
cm_scoped_token = response.json()

# Store scoped token for use in other notebooks
if input("Store scoped token for use in other notebooks? Y/n ").lower() == "y":
    %store cm_scoped_token

# Print scoped token for other uses
print(json.dumps(cm_scoped_token, indent=4))